## For specific zones at specific sites, return the run time of the RTU in each of the states (0-5) for a DR event date and a date with a comparable OAT that is not a DR event date.

## include table with both DR event date and Comparable date, OAT for both dates, and % of time in each state for each date, possibly include predicted energy use based on the RTU runtime and power of RTU in each state

In [1]:
import time
import datetime
import pytz
import pandas as pd

import os
import StringIO
import shutil
from shutil import copyfile

import urllib, json

from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
from numpy import trapz
import numpy as np

from matplotlib import style
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib import colors as mcolors
from matplotlib.colors import LinearSegmentedColormap
%matplotlib inline

import seaborn as sns
style.use("seaborn-notebook")
#from __future__ import division

from prettytable import PrettyTable


In [2]:
## query the zones at the specified site

## calling the hodclient

from xbos import get_client
from xbos.services.pundat import DataClient, timestamp, make_dataframe, merge_dfs
#from xbos.services.hod import HodClientHTTP
from xbos.services.hod import HodClient
from xbos.devices.thermostat import Thermostat
%matplotlib inline

## get specified site

c = get_client()
hod = HodClient("xbos/hod")
SITE = 'avenal-movie-theatre'


zones = """
SELECT ?zone FROM %s WHERE {
    ?zone rdf:type brick:HVAC_Zone .
};
"""

dateA = '2018-07-27'

res = hod.do_query(zones % SITE)
display (res)


Saw [xbos/hod] HodDB 1 seconds 285.807 ms ago


{'Count': 9,
 'Elapsed': 0,
 'Errors': None,
 'Rows': [{'?zone': 'HVAC_Zone_Theater_2'},
  {'?zone': 'HVAC_Zone_Theater_1'},
  {'?zone': 'HVAC_Zone_Room_D'},
  {'?zone': 'HVAC_Zone_Room_A'},
  {'?zone': 'HVAC_Zone_Pegasus_Hall'},
  {'?zone': 'HVAC_Zone_Outside'},
  {'?zone': 'HVAC_Zone_Main_Hallway'},
  {'?zone': 'HVAC_Zone_Lobby'},
  {'?zone': 'HVAC_Zone_Back_Hallway'}]}

In [3]:
## TSTAT State information
from xbos import get_client
from xbos.services.pundat import DataClient, timestamp, make_dataframe, merge_dfs
from xbos.services.hod import HodClientHTTP
from xbos.devices.thermostat import Thermostat

# get a bosswave client
c = get_client()
hod = HodClient("xbos/hod")
SITE = 'avenal-movie-theatre'

# query for TSAT Status
states = """
SELECT ?state ?state_uuid FROM %s WHERE {
    ?state rdf:type/rdfs:subClassOf* brick:Thermostat_Status . 
    ?state bf:uuid ?state_uuid .
};
"""

state_res = hod.do_query(states % SITE)
display (state_res)


Saw [xbos/hod] HodDB 1 seconds 419.353 ms ago


{'Count': 8,
 'Elapsed': 0,
 'Errors': None,
 'Rows': [{'?state': 'HVAC_Zone_Room_A_tstat_state',
   '?state_uuid': 'c9404f38-e36f-303d-a878-90209307bf28'},
  {'?state': 'HVAC_Zone_Theater_2_tstat_state',
   '?state_uuid': 'c085e817-5116-3c6f-ba9a-6882796ffa5f'},
  {'?state': 'HVAC_Zone_Pegasus_Hall_tstat_state',
   '?state_uuid': '8c252f6f-accf-3a28-ac71-18cd9e745272'},
  {'?state': 'HVAC_Zone_Theater_1_tstat_state',
   '?state_uuid': '6c8f111e-cc88-336a-8309-09d0c8b355b0'},
  {'?state': 'HVAC_Zone_Room_D_tstat_state',
   '?state_uuid': '1bbcfb99-98e3-3a40-a34e-94e41eb35c57'},
  {'?state': 'HVAC_Zone_Main_Hallway_tstat_state',
   '?state_uuid': 'c876fc25-2499-3357-9b94-387d5227b4a4'},
  {'?state': 'HVAC_Zone_Back_Hallway_tstat_state',
   '?state_uuid': '8285a29c-dcc5-305a-bb9d-8a2653173e75'},
  {'?state': 'HVAC_Zone_Lobby_tstat_state',
   '?state_uuid': '812903c9-624e-32e8-a7e1-bbf207c5eba0'}]}

In [4]:
## UUIDs for all sensors 

OAT = 'c3d0f042-2cb7-387c-8867-4190fef4fd2c'  #weather_KTLC1_temperature
#Demand = 'ea827a42-e6b9-3e0a-970f-c3a96f0ebea3'  #meter_0xd8d5b9000000b405
Pegasus_Hall_State = '8c252f6f-accf-3a28-ac71-18cd9e745272'  #HVAC_Zone_Pegasus_Hall_tstat_state
Room_A_State = 'c9404f38-e36f-303d-a878-90209307bf28'  #HVAC_Zone_Room_A_tstat_state
Room_D_State = '1bbcfb99-98e3-3a40-a34e-94e41eb35c57'  #HVAC_Zone_Room_D_tstat_state
Theater_1_State = '6c8f111e-cc88-336a-8309-09d0c8b355b0'  #HVAC_Zone_Theater_1_tstat_state
Theater_2_State = 'c085e817-5116-3c6f-ba9a-6882796ffa5f'  #HVAC_Zone_Theater_2_tstat_state


In [5]:
from xbos.services import mdal
client = mdal.MDALClient("xbos/mdal")
SITE = str(SITE) 


###Query for Tstat State data
queryEvent = {
    "Composition": [Pegasus_Hall_State, Room_A_State, Room_D_State, Theater_1_State, Theater_2_State], # mode and state at MAX AC-1
    "Selectors": [mdal.MAX, mdal.MAX, mdal.MAX, mdal.MAX, mdal.MAX], #mdal.{MAX, MIN, COUNT, MEAN, RAW} . Define one for each of the variables above
    "Variables": [
        {"Name": "Back_Hallway_State", ## 
         "Definition": "SELECT ?state_uuid FROM %s WHERE { ?state rdf:type/rdfs:subClassOf* brick:Thermostat_Status . ?state bf:uuid ?state_uuid . };" % SITE,
        },
        {"Name": "Room_A_State", ## 
         "Definition": "SELECT ?state_uuid FROM %s WHERE { ?state rdf:type/rdfs:subClassOf* brick:Thermostat_Status . ?state bf:uuid ?state_uuid . };" % SITE,
        },
        {"Name": "Room_D_State", ## 
         "Definition": "SELECT ?state_uuid FROM %s WHERE { ?state rdf:type/rdfs:subClassOf* brick:Thermostat_Status . ?state bf:uuid ?state_uuid . };" % SITE,
        },
        {"Name": "Theater_1_State", ## 
         "Definition": "SELECT ?state_uuid FROM %s WHERE { ?state rdf:type/rdfs:subClassOf* brick:Thermostat_Status . ?state bf:uuid ?state_uuid . };" % SITE,
        },
        {"Name": "Theater_2_State", ## 
         "Definition": "SELECT ?state_uuid FROM %s WHERE { ?state rdf:type/rdfs:subClassOf* brick:Thermostat_Status . ?state bf:uuid ?state_uuid . };" % SITE,
        },
    ],
    "Time": {
        "T0": "2018-07-27 14:00:00 PDT", ## "T0": 14:00:00 PDT",
        "T1": "2018-07-27 18:00:00 PDT", ## "T1": 18:00:00 PDT",
        "WindowSize": '1h', #1m, or 5m
        "Aligned": True,
    },
}
resp = client.do_query(queryEvent,timeout=300)
print resp.get('error') # see if there's an error
dfEvent = resp['df']
print dfEvent.describe()
display (dfEvent)

#date_Event = '07_10', '07_18', '08_02', '08_07'

###Query for Tstat State data
queryComp = {
    "Composition": [Pegasus_Hall_State, Room_A_State, Room_D_State, Theater_1_State, Theater_2_State], # mode and state at MAX AC-1
    "Selectors": [mdal.MAX, mdal.MAX, mdal.MAX, mdal.MAX, mdal.MAX], #mdal.{MAX, MIN, COUNT, MEAN, RAW} . Define one for each of the variables above
    "Variables": [
        {"Name": "Back_Hallway_State", ## 
         "Definition": "SELECT ?state_uuid FROM %s WHERE { ?state rdf:type/rdfs:subClassOf* brick:Thermostat_Status . ?state bf:uuid ?state_uuid . };" % SITE,
        },
        {"Name": "Room_A_State", ## 
         "Definition": "SELECT ?state_uuid FROM %s WHERE { ?state rdf:type/rdfs:subClassOf* brick:Thermostat_Status . ?state bf:uuid ?state_uuid . };" % SITE,
        },
        {"Name": "Room_D_State", ## 
         "Definition": "SELECT ?state_uuid FROM %s WHERE { ?state rdf:type/rdfs:subClassOf* brick:Thermostat_Status . ?state bf:uuid ?state_uuid . };" % SITE,
        },
        {"Name": "Theater_1_State", ## 
         "Definition": "SELECT ?state_uuid FROM %s WHERE { ?state rdf:type/rdfs:subClassOf* brick:Thermostat_Status . ?state bf:uuid ?state_uuid . };" % SITE,
        },
        {"Name": "Theater_2_State", ## 
         "Definition": "SELECT ?state_uuid FROM %s WHERE { ?state rdf:type/rdfs:subClassOf* brick:Thermostat_Status . ?state bf:uuid ?state_uuid . };" % SITE,
        },
    ],
    "Time": {
        "T0": "2018-07-18 14:00:00 PDT", ## "T0": 14:00:00 PDT",
        "T1": "2018-07-18 18:00:00 PDT", ## "T1": 18:00:00 PDT",
        "WindowSize": '1h', #1m, or 5m
        "Aligned": True,
    },
}
resp = client.do_query(queryComp,timeout=300)
print resp.get('error') # see if there's an error
dfComp = resp['df']
#print dfComp.describe()
#display (dfComp)


Saw [xbos/mdal] MDAL 3 seconds 795.741 ms ago
None
       8c252f6f-accf-3a28-ac71-18cd9e745272  \
count                                   4.0   
mean                                    0.0   
std                                     0.0   
min                                     0.0   
25%                                     0.0   
50%                                     0.0   
75%                                     0.0   
max                                     0.0   

       c9404f38-e36f-303d-a878-90209307bf28  \
count                                   4.0   
mean                                    2.0   
std                                     0.0   
min                                     2.0   
25%                                     2.0   
50%                                     2.0   
75%                                     2.0   
max                                     2.0   

       1bbcfb99-98e3-3a40-a34e-94e41eb35c57  \
count                                   4.0   
mean   

,8c252f6f-accf-3a28-ac71-18cd9e745272,c9404f38-e36f-303d-a878-90209307bf28,1bbcfb99-98e3-3a40-a34e-94e41eb35c57,6c8f111e-cc88-336a-8309-09d0c8b355b0,c085e817-5116-3c6f-ba9a-6882796ffa5f
2018-07-27 14:00:00-07:00,0.0,2.0,2.0,0.0,0.0
2018-07-27 15:00:00-07:00,0.0,2.0,2.0,0.0,0.0
2018-07-27 16:00:00-07:00,0.0,2.0,2.0,0.0,0.0
2018-07-27 17:00:00-07:00,0.0,2.0,2.0,0.0,0.0


None


In [6]:
from xbos.services import mdal
client = mdal.MDALClient("xbos/mdal")
SITE = str(SITE) 
Demand = 'ea827a42-e6b9-3e0a-970f-c3a96f0ebea3'  #meter_0xd8d5b9000000b405
Green_Btn = '68e04192-e924-36b8-9c5e-f072bd93ed07'  #green_button_meter_ekhaL7K1ZWy7yOfRpEIBOcC3v8Y=
###Query for Event Meter Demand
queryEventDemand = {
    "Composition": [Demand], # mode and state at MAX
    "Selectors": [mdal.MEAN], #mdal.{MAX, MIN, COUNT, MEAN, RAW} . Define one for each of the variables above
    "Variables": [
        {
         "Name": "demand",
         "Definition": "SELECT ?demand_uuid FROM %s WHERE { ?demand rdf:type/rdfs:subClassOf* brick:Electric_Meter . ?demand bf:uuid ?demand_uuid . };" % SITE,
        },
    ],
    "Time": {
        "T0": "2018-07-27 14:00:00 PDT",
        "T1": "2018-07-27 18:00:00 PDT",
        "WindowSize": '15m',
        "Aligned": True,
    },
}
resp = client.do_query(queryEventDemand,timeout=300)
print resp.get('error') # see if there's an error
event_demand = resp['df']
print event_demand.describe()

###Query for Baseline Meter Demand
queryBaselineDemand = {
    "Composition": [Green_Btn], # mode and state at MAX
    "Selectors": [mdal.MEAN], #mdal.{MAX, MIN, COUNT, MEAN, RAW} . Define one for each of the variables above
    "Variables": [
        {
         "Name": "green_btn",
         "Definition": "SELECT ?demand_uuid FROM %s WHERE { ?demand rdf:type/rdfs:subClassOf* brick:Electric_Meter . ?demand bf:uuid ?demand_uuid . };" % SITE,
        },
    ],
    "Time": {
        "T0": "2018-07-18 14:00:00 PDT",
        "T1": "2018-07-18 18:00:00 PDT",
        "WindowSize": '15m',
        "Aligned": True,
    },
}
resp = client.do_query(queryBaselineDemand,timeout=300)
print resp.get('error') # see if there's an error
baseline_demand = resp['df']
print baseline_demand.describe()

Saw [xbos/mdal] MDAL 4 seconds 393.69 ms ago
None
       ea827a42-e6b9-3e0a-970f-c3a96f0ebea3
count                             16.000000
mean                            4070.025983
std                              356.143286
min                             3514.035088
25%                             3737.314352
50%                             4124.603350
75%                             4329.440582
max                             4667.142857
None
       68e04192-e924-36b8-9c5e-f072bd93ed07
count                                   0.0
mean                                    NaN
std                                     NaN
min                                     NaN
25%                                     NaN
50%                                     NaN
75%                                     NaN
max                                     NaN


In [7]:
tstat_STATE = pd.DataFrame(list(state_res['Rows']))

display (tstat_STATE)

,?state,?state_uuid
0,HVAC_Zone_Room_A_tstat_state,c9404f38-e36f-303d-a878-90209307bf28
1,HVAC_Zone_Theater_2_tstat_state,c085e817-5116-3c6f-ba9a-6882796ffa5f
2,HVAC_Zone_Pegasus_Hall_tstat_state,8c252f6f-accf-3a28-ac71-18cd9e745272
3,HVAC_Zone_Theater_1_tstat_state,6c8f111e-cc88-336a-8309-09d0c8b355b0
4,HVAC_Zone_Room_D_tstat_state,1bbcfb99-98e3-3a40-a34e-94e41eb35c57
5,HVAC_Zone_Main_Hallway_tstat_state,c876fc25-2499-3357-9b94-387d5227b4a4
6,HVAC_Zone_Back_Hallway_tstat_state,8285a29c-dcc5-305a-bb9d-8a2653173e75
7,HVAC_Zone_Lobby_tstat_state,812903c9-624e-32e8-a7e1-bbf207c5eba0


In [8]:
##Sort column ?state to get the right order of tstat names
tstat_STATE.sort_values(by=['?state'], inplace=True, ascending = True)
tstat_STATE = tstat_STATE.reset_index(drop=True)

display (tstat_STATE)

,?state,?state_uuid
0,HVAC_Zone_Back_Hallway_tstat_state,8285a29c-dcc5-305a-bb9d-8a2653173e75
1,HVAC_Zone_Lobby_tstat_state,812903c9-624e-32e8-a7e1-bbf207c5eba0
2,HVAC_Zone_Main_Hallway_tstat_state,c876fc25-2499-3357-9b94-387d5227b4a4
3,HVAC_Zone_Pegasus_Hall_tstat_state,8c252f6f-accf-3a28-ac71-18cd9e745272
4,HVAC_Zone_Room_A_tstat_state,c9404f38-e36f-303d-a878-90209307bf28
5,HVAC_Zone_Room_D_tstat_state,1bbcfb99-98e3-3a40-a34e-94e41eb35c57
6,HVAC_Zone_Theater_1_tstat_state,6c8f111e-cc88-336a-8309-09d0c8b355b0
7,HVAC_Zone_Theater_2_tstat_state,c085e817-5116-3c6f-ba9a-6882796ffa5f


In [9]:
##Select items based on location when they are non-consecutive
zone_list = tstat_STATE.loc[[3, 4, 5, 6, 7]] #[16, 3, 0, 5, 12] for CSU-DH
display (zone_list)

,?state,?state_uuid
3,HVAC_Zone_Pegasus_Hall_tstat_state,8c252f6f-accf-3a28-ac71-18cd9e745272
4,HVAC_Zone_Room_A_tstat_state,c9404f38-e36f-303d-a878-90209307bf28
5,HVAC_Zone_Room_D_tstat_state,1bbcfb99-98e3-3a40-a34e-94e41eb35c57
6,HVAC_Zone_Theater_1_tstat_state,6c8f111e-cc88-336a-8309-09d0c8b355b0
7,HVAC_Zone_Theater_2_tstat_state,c085e817-5116-3c6f-ba9a-6882796ffa5f


In [10]:
##Reorder columns 
uuid_zone = zone_list[['?state_uuid', '?state']] #
display (uuid_zone)


,?state_uuid,?state
3,8c252f6f-accf-3a28-ac71-18cd9e745272,HVAC_Zone_Pegasus_Hall_tstat_state
4,c9404f38-e36f-303d-a878-90209307bf28,HVAC_Zone_Room_A_tstat_state
5,1bbcfb99-98e3-3a40-a34e-94e41eb35c57,HVAC_Zone_Room_D_tstat_state
6,6c8f111e-cc88-336a-8309-09d0c8b355b0,HVAC_Zone_Theater_1_tstat_state
7,c085e817-5116-3c6f-ba9a-6882796ffa5f,HVAC_Zone_Theater_2_tstat_state


In [11]:
list(uuid_zone.itertuples(index=False, name=None))

[('8c252f6f-accf-3a28-ac71-18cd9e745272',
  'HVAC_Zone_Pegasus_Hall_tstat_state'),
 ('c9404f38-e36f-303d-a878-90209307bf28', 'HVAC_Zone_Room_A_tstat_state'),
 ('1bbcfb99-98e3-3a40-a34e-94e41eb35c57', 'HVAC_Zone_Room_D_tstat_state'),
 ('6c8f111e-cc88-336a-8309-09d0c8b355b0', 'HVAC_Zone_Theater_1_tstat_state'),
 ('c085e817-5116-3c6f-ba9a-6882796ffa5f', 'HVAC_Zone_Theater_2_tstat_state')]

In [12]:
uuid_zone.set_index('?state_uuid').T.to_dict('list') # why is list unsorted .sort_values('?state', ascending=True)


{'1bbcfb99-98e3-3a40-a34e-94e41eb35c57': ['HVAC_Zone_Room_D_tstat_state'],
 '6c8f111e-cc88-336a-8309-09d0c8b355b0': ['HVAC_Zone_Theater_1_tstat_state'],
 '8c252f6f-accf-3a28-ac71-18cd9e745272': ['HVAC_Zone_Pegasus_Hall_tstat_state'],
 'c085e817-5116-3c6f-ba9a-6882796ffa5f': ['HVAC_Zone_Theater_2_tstat_state'],
 'c9404f38-e36f-303d-a878-90209307bf28': ['HVAC_Zone_Room_A_tstat_state']}

In [13]:
##Rename columns as zones, based on UUID 
col_names = {'index':'DateTime', 
             '1bbcfb99-98e3-3a40-a34e-94e41eb35c57': 'HVAC_Zone_Room_D_tstat_state', 
             '6c8f111e-cc88-336a-8309-09d0c8b355b0': 'HVAC_Zone_Theater_1_tstat_state',
             '8c252f6f-accf-3a28-ac71-18cd9e745272': 'HVAC_Zone_Pegasus_Hall_tstat_state',
             'c085e817-5116-3c6f-ba9a-6882796ffa5f': 'HVAC_Zone_Theater_2_tstat_state',
             'c9404f38-e36f-303d-a878-90209307bf28': 'HVAC_Zone_Room_A_tstat_state'
} #'index':'DateTime'

In [14]:
comp_dates = ['8/8/18', '7/5/18', '7/26/18', '6/14/18', '10/16/18', 
              '10/31/18', '7/25/18', '7/5/18', '9/14/18', '10/1/18', '9/5/18', '7/25/18']
cpp_dates = ['7/6/18', '7/9/18', '7/10/18', '7/17/18', '7/18/18', 
             '8/1/18', '8/2/18', '8/6/18', '8/7/18', '8/9/18', '9/28/18', '10/18/18']

date_Event = '07_19' #'07_06', '07_10', '07_18', '08_02', '08_07', '09_28'
date_Comp = '08_01' #'08_08', '07_26', '10_16', '07_25', '09_14', '09_05'


In [15]:
##Reset the index of df (only works one DF at a time?)

df_Event = dfEvent.reset_index(drop=False) #'DateTime'
df_Comp = dfComp.reset_index(drop=False) #'DateTime'
display (df_Event)

,index,8c252f6f-accf-3a28-ac71-18cd9e745272,c9404f38-e36f-303d-a878-90209307bf28,1bbcfb99-98e3-3a40-a34e-94e41eb35c57,6c8f111e-cc88-336a-8309-09d0c8b355b0,c085e817-5116-3c6f-ba9a-6882796ffa5f
0,2018-07-27 14:00:00-07:00,0.0,2.0,2.0,0.0,0.0
1,2018-07-27 15:00:00-07:00,0.0,2.0,2.0,0.0,0.0
2,2018-07-27 16:00:00-07:00,0.0,2.0,2.0,0.0,0.0
3,2018-07-27 17:00:00-07:00,0.0,2.0,2.0,0.0,0.0


In [16]:
##Rename the columns 

for df in df_Event, df_Comp:
    df.rename(columns=col_names, inplace=True)
    display (df_Comp)

,index,8c252f6f-accf-3a28-ac71-18cd9e745272,c9404f38-e36f-303d-a878-90209307bf28,1bbcfb99-98e3-3a40-a34e-94e41eb35c57,6c8f111e-cc88-336a-8309-09d0c8b355b0,c085e817-5116-3c6f-ba9a-6882796ffa5f
0,2018-07-18 14:00:00-07:00,2.0,0.0,0.0,0.0,0.0
1,2018-07-18 15:00:00-07:00,2.0,0.0,0.0,0.0,0.0
2,2018-07-18 16:00:00-07:00,5.0,0.0,0.0,0.0,0.0
3,2018-07-18 17:00:00-07:00,2.0,0.0,0.0,0.0,0.0


,DateTime,HVAC_Zone_Pegasus_Hall_tstat_state,HVAC_Zone_Room_A_tstat_state,HVAC_Zone_Room_D_tstat_state,HVAC_Zone_Theater_1_tstat_state,HVAC_Zone_Theater_2_tstat_state
0,2018-07-18 14:00:00-07:00,2.0,0.0,0.0,0.0,0.0
1,2018-07-18 15:00:00-07:00,2.0,0.0,0.0,0.0,0.0
2,2018-07-18 16:00:00-07:00,5.0,0.0,0.0,0.0,0.0
3,2018-07-18 17:00:00-07:00,2.0,0.0,0.0,0.0,0.0


In [17]:
##Create date and time columns and separate 'YYYY-MM-DD' from 'HH:MM'
for df in df_Event, df_Comp:
    df['YYYY-MM-DD'] = [x.date() for x in df_Event['DateTime']]
    df['HH:MM'] = [x.time() for x in df_Event['DateTime']]
    display (df_Comp)


,DateTime,HVAC_Zone_Pegasus_Hall_tstat_state,HVAC_Zone_Room_A_tstat_state,HVAC_Zone_Room_D_tstat_state,HVAC_Zone_Theater_1_tstat_state,HVAC_Zone_Theater_2_tstat_state
0,2018-07-18 14:00:00-07:00,2.0,0.0,0.0,0.0,0.0
1,2018-07-18 15:00:00-07:00,2.0,0.0,0.0,0.0,0.0
2,2018-07-18 16:00:00-07:00,5.0,0.0,0.0,0.0,0.0
3,2018-07-18 17:00:00-07:00,2.0,0.0,0.0,0.0,0.0


,DateTime,HVAC_Zone_Pegasus_Hall_tstat_state,HVAC_Zone_Room_A_tstat_state,HVAC_Zone_Room_D_tstat_state,HVAC_Zone_Theater_1_tstat_state,HVAC_Zone_Theater_2_tstat_state,YYYY-MM-DD,HH:MM
0,2018-07-18 14:00:00-07:00,2.0,0.0,0.0,0.0,0.0,2018-07-27,14:00:00
1,2018-07-18 15:00:00-07:00,2.0,0.0,0.0,0.0,0.0,2018-07-27,15:00:00
2,2018-07-18 16:00:00-07:00,5.0,0.0,0.0,0.0,0.0,2018-07-27,16:00:00
3,2018-07-18 17:00:00-07:00,2.0,0.0,0.0,0.0,0.0,2018-07-27,17:00:00


In [18]:
##Turn date and time columns in to str

for df in df_Event, df_Comp:
    df['YYYY-MM-DD'] = df['YYYY-MM-DD'].astype(str)
    df['HH:MM'] = df['HH:MM'].astype(str)
    display (df_Event)



,DateTime,HVAC_Zone_Pegasus_Hall_tstat_state,HVAC_Zone_Room_A_tstat_state,HVAC_Zone_Room_D_tstat_state,HVAC_Zone_Theater_1_tstat_state,HVAC_Zone_Theater_2_tstat_state,YYYY-MM-DD,HH:MM
0,2018-07-27 14:00:00-07:00,0.0,2.0,2.0,0.0,0.0,2018-07-27,14:00:00
1,2018-07-27 15:00:00-07:00,0.0,2.0,2.0,0.0,0.0,2018-07-27,15:00:00
2,2018-07-27 16:00:00-07:00,0.0,2.0,2.0,0.0,0.0,2018-07-27,16:00:00
3,2018-07-27 17:00:00-07:00,0.0,2.0,2.0,0.0,0.0,2018-07-27,17:00:00


,DateTime,HVAC_Zone_Pegasus_Hall_tstat_state,HVAC_Zone_Room_A_tstat_state,HVAC_Zone_Room_D_tstat_state,HVAC_Zone_Theater_1_tstat_state,HVAC_Zone_Theater_2_tstat_state,YYYY-MM-DD,HH:MM
0,2018-07-27 14:00:00-07:00,0.0,2.0,2.0,0.0,0.0,2018-07-27,14:00:00
1,2018-07-27 15:00:00-07:00,0.0,2.0,2.0,0.0,0.0,2018-07-27,15:00:00
2,2018-07-27 16:00:00-07:00,0.0,2.0,2.0,0.0,0.0,2018-07-27,16:00:00
3,2018-07-27 17:00:00-07:00,0.0,2.0,2.0,0.0,0.0,2018-07-27,17:00:00


In [19]:
Event = 'df_'+str(date_Event)
Comp = 'df_'+str(date_Comp)
print Comp

df_08_01


In [20]:
##Reorder columns 

Event = df_Event.iloc[:, np.r_[7, 1:6]] #[['HH:MM', 'State1 (09-28)']] # DateTime, YYYY-MM-DD
Comp = df_Comp.iloc[:, np.r_[7, 1:6]] # DateTime, YYYY-MM-DD# 'State3 (09-11)'

display (Event)

,HH:MM,HVAC_Zone_Pegasus_Hall_tstat_state,HVAC_Zone_Room_A_tstat_state,HVAC_Zone_Room_D_tstat_state,HVAC_Zone_Theater_1_tstat_state,HVAC_Zone_Theater_2_tstat_state
0,14:00:00,0.0,2.0,2.0,0.0,0.0
1,15:00:00,0.0,2.0,2.0,0.0,0.0
2,16:00:00,0.0,2.0,2.0,0.0,0.0
3,17:00:00,0.0,2.0,2.0,0.0,0.0


In [21]:
##Fill NaN values to find missing data

for df in Event, Comp:
    df.fillna(value=99, inplace=True) # use value 99 to find NaN
    display (df)

/Applications/anaconda/lib/python2.7/site-packages/pandas/core/frame.py:3790: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


,HH:MM,HVAC_Zone_Pegasus_Hall_tstat_state,HVAC_Zone_Room_A_tstat_state,HVAC_Zone_Room_D_tstat_state,HVAC_Zone_Theater_1_tstat_state,HVAC_Zone_Theater_2_tstat_state
0,14:00:00,0.0,2.0,2.0,0.0,0.0
1,15:00:00,0.0,2.0,2.0,0.0,0.0
2,16:00:00,0.0,2.0,2.0,0.0,0.0
3,17:00:00,0.0,2.0,2.0,0.0,0.0


,HH:MM,HVAC_Zone_Pegasus_Hall_tstat_state,HVAC_Zone_Room_A_tstat_state,HVAC_Zone_Room_D_tstat_state,HVAC_Zone_Theater_1_tstat_state,HVAC_Zone_Theater_2_tstat_state
0,14:00:00,2.0,0.0,0.0,0.0,0.0
1,15:00:00,2.0,0.0,0.0,0.0,0.0
2,16:00:00,5.0,0.0,0.0,0.0,0.0
3,17:00:00,2.0,0.0,0.0,0.0,0.0


In [22]:
##A list of column headers if needed

print Comp.columns.values.tolist()

['HH:MM', 'HVAC_Zone_Pegasus_Hall_tstat_state', 'HVAC_Zone_Room_A_tstat_state', 'HVAC_Zone_Room_D_tstat_state', 'HVAC_Zone_Theater_1_tstat_state', 'HVAC_Zone_Theater_2_tstat_state']


In [23]:
##Reset index to HH:MM
Event.set_index('HH:MM', inplace=True) # DateTime, YYYY-MM-DD
Comp.set_index('HH:MM', inplace=True) # DateTime, YYYY-MM-DD


In [24]:
##Count unique elements for Baseline and DR Event days
unique_elements, counts_elements = np.unique(Comp, return_counts=True)
print(np.asarray((unique_elements, counts_elements)))

[[ 0.  2.  5.]
 [16.  3.  1.]]


In [25]:
##Count the number of lines in each of the dfs (1440 minutes in a 24 hour period)
Comp.count()

HVAC_Zone_Pegasus_Hall_tstat_state    4
HVAC_Zone_Room_A_tstat_state          4
HVAC_Zone_Room_D_tstat_state          4
HVAC_Zone_Theater_1_tstat_state       4
HVAC_Zone_Theater_2_tstat_state       4
dtype: int64

In [26]:
##Count number of hours in each state category
Event = Event.apply(lambda y: y.value_counts(sort=False))
Event.index = Event.index.astype(str)
display (Event)

Event.to_csv('df_'+str(date_Event)+'.csv')

,HVAC_Zone_Pegasus_Hall_tstat_state,HVAC_Zone_Room_A_tstat_state,HVAC_Zone_Room_D_tstat_state,HVAC_Zone_Theater_1_tstat_state,HVAC_Zone_Theater_2_tstat_state
0.0,4.0,NaN,NaN,4.0,4.0
2.0,NaN,4.0,4.0,NaN,NaN


In [27]:
##Count number of hours in each state category
Comp = Comp.apply(lambda y: y.value_counts(sort=False))
Comp.index = Comp.index.astype(str)
display (Comp)

Comp.to_csv('df_'+str(date_Comp)+'.csv')

,HVAC_Zone_Pegasus_Hall_tstat_state,HVAC_Zone_Room_A_tstat_state,HVAC_Zone_Room_D_tstat_state,HVAC_Zone_Theater_1_tstat_state,HVAC_Zone_Theater_2_tstat_state
0.0,NaN,4.0,4.0,4.0,4.0
2.0,3.0,NaN,NaN,NaN,NaN
5.0,1.0,NaN,NaN,NaN,NaN


##Replace NaN here with Zero (0) to prevent over counting at analysis stage!!!

In [28]:
##Fill NaN values to find missing data

for df in Event, Comp:
    df.fillna(value=0, inplace=True) # use value 99 to find NaN 
    display (df)

,HVAC_Zone_Pegasus_Hall_tstat_state,HVAC_Zone_Room_A_tstat_state,HVAC_Zone_Room_D_tstat_state,HVAC_Zone_Theater_1_tstat_state,HVAC_Zone_Theater_2_tstat_state
0.0,4.0,0.0,0.0,4.0,4.0
2.0,0.0,4.0,4.0,0.0,0.0


,HVAC_Zone_Pegasus_Hall_tstat_state,HVAC_Zone_Room_A_tstat_state,HVAC_Zone_Room_D_tstat_state,HVAC_Zone_Theater_1_tstat_state,HVAC_Zone_Theater_2_tstat_state
0.0,0.0,4.0,4.0,4.0,4.0
2.0,3.0,0.0,0.0,0.0,0.0
5.0,1.0,0.0,0.0,0.0,0.0


In [29]:
Eventa = 'df_'+str(date_Event)+'test'
Compa = 'df_'+str(date_Comp)+'test'

print Eventa

df_07_19test


In [30]:
##reset index of new dfs to groupby and plot 
Eventa = Event.reset_index(drop=False)
Compa = Comp.reset_index(drop=False)
display (Compa)

,index,HVAC_Zone_Pegasus_Hall_tstat_state,HVAC_Zone_Room_A_tstat_state,HVAC_Zone_Room_D_tstat_state,HVAC_Zone_Theater_1_tstat_state,HVAC_Zone_Theater_2_tstat_state
0,0.0,0.0,4.0,4.0,4.0,4.0
1,2.0,3.0,0.0,0.0,0.0,0.0
2,5.0,1.0,0.0,0.0,0.0,0.0


In [31]:
newcol_names = {'index':'State', 
                'HVAC_Zone_Pegasus_Hall_tstat_state': 'Pegasus_Hall', 
                'HVAC_Zone_Room_A_tstat_state': 'Room_A', 
                'HVAC_Zone_Room_D_tstat_state': 'Room_D', 
                'HVAC_Zone_Theater_1_tstat_state': 'Theater_1', 
                'HVAC_Zone_Theater_2_tstat_state': 'Theater_2'
} #'index':'DateTime'


In [32]:
##Rename new columns for plotting
Eventa.rename(columns = newcol_names, inplace=True)
Compa.rename(columns = newcol_names, inplace=True)

#df = df.apply(pd.to_numeric)

#df_09_28new = df_09_28test.rename(columns=newcol_names, inplace=True)
display (Eventa)

,State,Pegasus_Hall,Room_A,Room_D,Theater_1,Theater_2
0,0.0,4.0,0.0,0.0,4.0,4.0
1,2.0,0.0,4.0,4.0,0.0,0.0


In [33]:
##Make all numbers int
Eventa = Eventa.apply(pd.to_numeric, errors='ignore').astype(int)
Compa = Compa.apply(pd.to_numeric, errors='ignore').astype(int)
#display (df_10_02test)
Compa

,State,Pegasus_Hall,Room_A,Room_D,Theater_1,Theater_2
0,0,0,4,4,4,4
1,2,3,0,0,0,0
2,5,1,0,0,0,0


In [34]:
print Eventa.columns.values.tolist()

['State', 'Pegasus_Hall', 'Room_A', 'Room_D', 'Theater_1', 'Theater_2']


In [35]:
##Add new rows and sort

column_head = Eventa.columns.values.tolist()

s0 = pd.Series([0.0, 0, 0, 0, 0, 0])
s1 = pd.Series([1.0, 0, 0, 0, 0, 0])
s2 = pd.Series([2.0, 0, 0, 0, 0, 0])
s3 = pd.Series([3.0, 0, 0, 0, 0, 0])
s4 = pd.Series([4.0, 0, 0, 0, 0, 0])
s5 = pd.Series([5.0, 0, 0, 0, 0, 0])
s6 = pd.Series([99.0, 0, 0, 0, 0, 0])

df_3rows = pd.DataFrame([list(s1), list(s2), list(s3)],  columns = column_head).astype(int)
df_4rows = pd.DataFrame([list(s1), list(s2), list(s3), list(s4)],  columns = column_head).astype(int)
df_6rows = pd.DataFrame([list(s0), list(s1), list(s2), list(s3), list(s4), list(s5), list(s6)],  columns = column_head).astype(int)

display (df_6rows)

,State,Pegasus_Hall,Room_A,Room_D,Theater_1,Theater_2
0,0,0,0,0,0,0
1,1,0,0,0,0,0
2,2,0,0,0,0,0
3,3,0,0,0,0,0
4,4,0,0,0,0,0
5,5,0,0,0,0,0
6,99,0,0,0,0,0


In [36]:
fin_Eventa = 'df_'+str(date_Event)+'_final'
fin_Compa = 'df_'+str(date_Comp)+'_final'

print fin_Eventa

df_07_19_final


In [37]:
##Merge new dfs to make a final df  

fin_Eventa = df_6rows.merge(Eventa, on=list(df_4rows), how='outer')
fin_Eventa.drop_duplicates(subset='State', inplace=True, keep='last')
fin_Eventa.sort_values('State', inplace=True)
fin_Eventa = fin_Eventa.reset_index(drop=True)
display (fin_Eventa)

,State,Pegasus_Hall,Room_A,Room_D,Theater_1,Theater_2
0,0,4,0,0,4,4
1,1,0,0,0,0,0
2,2,0,4,4,0,0
3,3,0,0,0,0,0
4,4,0,0,0,0,0
5,5,0,0,0,0,0
6,99,0,0,0,0,0


In [38]:
##Merge new dfs to make a final df  

fin_Compa = df_6rows.merge(Compa, on=list(df_4rows), how='outer')
fin_Compa.drop_duplicates(subset='State', inplace=True, keep='last')
fin_Compa.sort_values('State', inplace=True)
fin_Compa = fin_Compa.reset_index(drop=True)
display (fin_Compa)



,State,Pegasus_Hall,Room_A,Room_D,Theater_1,Theater_2
0,0,0,4,4,4,4
1,1,0,0,0,0,0
2,2,3,0,0,0,0
3,3,0,0,0,0,0
4,4,0,0,0,0,0
5,5,1,0,0,0,0
6,99,0,0,0,0,0


##Start of New Analysis Code

In [39]:
#Baseline 1-Stage Clg (Hrs) & #Baseline 2-Stage Clg (Hrs)

    ##Reindex DF & rename rows
baseline_runtime = fin_Compa.iloc[[0,2,5,6]] #select only the cooling stages [1:-1]
baseline_runtime.rename({0: 'Baseline Off (Hrs)', 
                         2: 'Baseline 1-Stage Clg (Hrs)', 
                         5: 'Baseline 2-Stage Clg (Hrs)', 
                         6: 'Baseline Missing Data (Hrs)'}, axis='index', inplace=True)
baseline_runtime.drop('State', axis=1, inplace=True)
#display(baseline_runtime)

#DR Event 1-Stage Clg (Hrs) & #DR Event 2-Stage Clg (Hrs)

    ##Reindex DF & rename rows
event_runtime = fin_Eventa.iloc[[0,2,5,6]] #select only the cooling stages [1:-1]
event_runtime.rename({0: 'DR Event Off (Hrs)', 
                      2: 'DR Event 1-Stage Clg (Hrs)', 
                      5: 'DR Event 2-Stage Clg (Hrs)', 
                      6: 'DR Event Missing Data (Hrs)'}, axis='index', inplace=True)
event_runtime.drop('State', axis=1, inplace=True)
#display(event_runtime)

site_runtime = pd.concat([baseline_runtime, event_runtime], join_axes=[baseline_runtime.columns])
    
#Adjusted Baseline & DR Event Runtime ((Stage-1) + (2 * Stage-2))

site_runtime.loc['Baseline Adjusted Runtime',:] = site_runtime.loc['Baseline 2-Stage Clg (Hrs)',:].mul(2) + site_runtime.loc['Baseline 1-Stage Clg (Hrs)',:]
site_runtime.loc['DR Event Adjusted Runtime',:] = site_runtime.loc['DR Event 2-Stage Clg (Hrs)',:].mul(2) + site_runtime.loc['DR Event 1-Stage Clg (Hrs)',:]

## Round Numbers by row? or at the end?
# Ratio['Baseline'] = pd.Series([round(val, 1) for val in Ratio['Baseline']], index = Ratio.index)
# Ratio['DR Event'] = pd.Series([round(val, 1) for val in Ratio['DR Event']], index = Ratio.index)

##Ratio (DR Event/Baseline) - calculated
site_runtime.loc['Ratio',:] = (site_runtime.loc['DR Event Adjusted Runtime',:]/site_runtime.loc['Baseline Adjusted Runtime',:]).round(2)

##Savings ((Baseline - DR Event) / Baseline), Calculate % savings between Event day and Comp Day runtimes
site_runtime.loc['Savings',:] = ((site_runtime.loc['Baseline Adjusted Runtime',:] - site_runtime.loc['DR Event Adjusted Runtime',:])/site_runtime.loc['Baseline Adjusted Runtime',:]).apply('{:.0%}'.format)

##Get peak demand for stage 1 and stage 2 cooling from DF

##Read Excel with building system data
filepath = '/Users/gregthomson/Box Sync/Sponsor Awards/CEC 84748 XBOS-DR/CEC XBOS-DR/Project Management/Potential building sites/DR_Planning_Docs/Test_Python/XBOS_DR_Runtime_SiteZone_Base.xlsx'
#XBOS_DR_Runtime_SiteZone = pd.read_excel(filepath)
#print XBOS_DR_Runtime_SiteZone.columns
cols_to_drop = ['Htg-S1 Power', 'Htg-S2 Power', 'Clg-S1 Power', 'Clg-S2 Power', 'Fan Power']
controlled_zones = pd.read_excel(filepath)
controlled_zones.drop(cols_to_drop, axis=1, inplace=True)

##Sort for zones controlled during dr events
controlled_zones = controlled_zones.loc[controlled_zones['Controllable Zone (0=False, 1=True)'] == 1]

##Reset index of DF and drop unneccessary columns
rtu_power = controlled_zones.reset_index(inplace=False, drop=True)
cols_to_drop2 = ['Controllable Zone (0=False, 1=True)', 'Occupancy Sensor (0=False, 1=True)', 'Occupancy Sensor Permanent (0=False, 1=True)', 'Heating Stages', 'Cooling Stages', 'Fan Stages']
rtu_power.drop(cols_to_drop2, axis=1, inplace=True) 

##Keep only data for zones from filter
zone_filter = rtu_power
cols_to_keep = ['HVAC_Zone_Pegasus_Hall', 
                'HVAC_Zone_Room_A', 
                'HVAC_Zone_Room_D', 
                'HVAC_Zone_Theater_1', 
                'HVAC_Zone_Theater_2']
zones = zone_filter[zone_filter['Zone'].isin(cols_to_keep)]

##Reorder the columns
frame = zones.T
frame = frame[[9, 10, 11, 7, 8]]

##Extract cooling stage peak demand
stage_cooling = frame
stage_demand = stage_cooling.drop(stage_cooling.index[[1, 2, 5, 6, 7, 8]]) #, inplace=True

##Reset index
demand = stage_demand.T

##Strip HVAC_Zone_ characters from the Zone column
demand['Zone'] = demand['Zone'].str.replace('HVAC_Zone_', '')
zone_list2 = demand['Zone'].tolist()

##Rename column headers
df_peak_demand = demand.T
df_peak_demand.columns = zone_list2
df_peak_demand = df_peak_demand.reset_index() #drop=True
df_peak_demand = df_peak_demand.set_index('index', drop=True)#, inplace=True)

site_runtime.loc['Stage-1 Clg Demand (kW)',:] = df_peak_demand.loc['Clg-S1 kW']#['Clg-S1 kW']
site_runtime.loc['Stage-2 Clg Demand (kW)',:] = df_peak_demand.loc['Clg-S2 kW']#['Clg-S1 kW']

##Calculate total energy consumption from Runtime * Peak Demand (in Hours) for whole day
##Make sure these calculations are using the Peak Demand for appropriate zone
##Calculation (S1=(runtime*peak demand) + S2 = (runtime * (2*peak demand)))

#Zone Runtime kWh (Baseline)
site_runtime.loc['Zone Runtime kWh (Baseline)',:] = (site_runtime.iloc[1,0:].mul(site_runtime.iloc[12,0:]))+2*(site_runtime.iloc[2,0:].mul(site_runtime.iloc[13,0:]))

#Zone Runtime kWh (DR Event)
site_runtime.loc['Zone Runtime kWh (DR Event)',:] = (site_runtime.iloc[5,0:].mul(site_runtime.iloc[12,0:]))+2*(site_runtime.iloc[6,0:].mul(site_runtime.iloc[13,0:]))

#Zone Runtime kWh Savings (Baseline - DR Event)
site_runtime.loc['Zone Runtime kWh Savings (Baseline - DR Event)',:] = (site_runtime.loc['Zone Runtime kWh (Baseline)',:])-(site_runtime.loc['Zone Runtime kWh (DR Event)',:])

#Zone Area (SF)
#Zone Volume (CF)
#Net Area All Zones (GSF)

##Get Zone areas, etc., only use the index, zone, zone area, and zone volume columns
XBOS_DR_ZoneEUI = pd.read_excel(filepath, sheet_name='Sites_Zones', usecols='A,B,S:V')

zone_list = cols_to_keep
site_select = XBOS_DR_ZoneEUI.loc[XBOS_DR_ZoneEUI['Zone'].isin(zone_list)]
site_zones = site_select.reset_index(drop=True) #inplace=False

##Strip HVAC_Zone_ characters from the Zone column
site_zones['Zone'] = site_zones['Zone'].str.replace('HVAC_Zone_', '')

##Rename column headers
df_site_zones = site_zones.T
zones = df_site_zones.values.tolist()[0]
df_site_zones.columns = zones
df_site_zones = df_site_zones.reset_index() #drop=True
df_site_zones = df_site_zones.set_index('index', drop=True)#, inplace=True)

##Get Building GSF
site = SITE #'avenal-movie-theatre' #use SITE from HOD/MDAL
site_GSF = XBOS_DR_ZoneEUI.groupby(XBOS_DR_ZoneEUI.index).get_group(site)
GSF = site_GSF.loc[site_GSF['Zone'] == 'HVAC_Zone_Lobby', 'Net Area All Zones'].iloc[0]
multiplier = site_GSF.loc[site_GSF['Zone'] == 'HVAC_Zone_Lobby', 'Meter Multiplier'].iloc[0]

##Fill in all NaNs with net area
value = GSF
multiplier_value = multiplier #meter multiplier
df_site_zones.loc['Net Area All Zones'] = df_site_zones.loc['Net Area All Zones'].fillna(value)

##Append Zone information to DF
site_runtime.loc['Zone Area'] = (df_site_zones.iloc[1,0:])
site_runtime.loc['Zone Volume'] = (df_site_zones.iloc[2,0:])
site_runtime.loc['Net Area All Zones'] = (df_site_zones.iloc[3,0:])
site_runtime.loc['Meter Multiplier'] = multiplier_value

#Building Event Energy (kWh)

##Reset the index of df
eagle_data = event_demand.reset_index(drop=False) #'DateTime'

##Rename the columns 
new_cols = {'index':'DateTime', 'ea827a42-e6b9-3e0a-970f-c3a96f0ebea3':'Demand (W)'}
eagle_data2 = eagle_data.rename(columns=new_cols)

##Get Site Meter Multiplier

##Fill in all NaNs with multiplier value
df_site_zones.loc['Meter Multiplier'] = df_site_zones.loc['Meter Multiplier'].fillna(multiplier_value)

##Calculate energy kWh from demand W
event_energy = eagle_data2
event_energy['Meter Multiplier'] = multiplier_value
event_energy['Event Energy (kWh)'] = (((eagle_data2.iloc[:, 1]*multiplier_value )/ 1000) * 0.25)

##Sum total energy for window of analysis
energy = event_energy['Event Energy (kWh)'].sum()

##Event day energy total 
site_runtime.loc['Building Event Energy (kWh)'] = energy.round(2) #event_energy['Event Energy (kWh)'].sum()

#Zone EUI (kWh/SF)
##Calculate the EUI for the each zone (Zone Energy/Zone Area), then Round numbers
site_runtime.loc['Zone EUI (kWh/SF)'] = (site_runtime.loc['Zone Runtime kWh (DR Event)'] / site_runtime.loc['Zone Area'])#.round(4)

#Building EUI (kWh/SF)
##Calculate the Event day EUI for the whole building (Bldg Evnt Energy/Bldg Area), then Round numbers
site_runtime.loc['Building EUI (kWh/SF)'] = (site_runtime.loc['Building Event Energy (kWh)'] / site_runtime.loc['Net Area All Zones'])#.round(4)

#Zone SF % of GSF
##Calculate the zone area as a percent of total GSF, then convert to percent
site_runtime.loc['Zone SF % of GSF'] = (site_runtime.loc['Zone Area'] / site_runtime.loc['Net Area All Zones']).apply('{:.0%}'.format)

#Zone Energy % of Building Energy
##Zone Energy as % of Bldg Energy
site_runtime.loc['Zone Energy % of Building Energy'] = (site_runtime.loc['Zone Runtime kWh (DR Event)'] / site_runtime.loc['Building Event Energy (kWh)']).apply('{:.0%}'.format)

#Zone CO2 savings
##Zone Energy Savings * PGE Emissions Factor
##PGE emissions for electricity: 0.524 lbs CO2/kWh 
##National Average: 1,559 lbs CO2/MWh × (5 × 10-4 tons/lb) × 0.001 MWh/kWh = 7.795 × 10-4 tons CO2/kWh
##National Average: 1,559 lbs CO2/MWh × 0.001 MWh/kWh = 1.559 lbs CO2/kWh
site_runtime.loc['GHG Emissions Reductions (lbs-CO2/event)'] = (site_runtime.loc['Zone Runtime kWh Savings (Baseline - DR Event)'] * 0.524) #

display(site_runtime)

/Applications/anaconda/lib/python2.7/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
/Applications/anaconda/lib/python2.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,Pegasus_Hall,Room_A,Room_D,Theater_1,Theater_2
Baseline Off (Hrs),0,4,4,4,4
Baseline 1-Stage Clg (Hrs),3,0,0,0,0
Baseline 2-Stage Clg (Hrs),1,0,0,0,0
Baseline Missing Data (Hrs),0,0,0,0,0
DR Event Off (Hrs),4,0,0,4,4
DR Event 1-Stage Clg (Hrs),0,4,4,0,0
DR Event 2-Stage Clg (Hrs),0,0,0,0,0
DR Event Missing Data (Hrs),0,0,0,0,0
Baseline Adjusted Runtime,5,0,0,0,0
DR Event Adjusted Runtime,0,4,4,0,0


In [40]:
event_report = site_runtime
event_report.to_csv(str(site)+'_'+str(date_Comp)+'-'+str(date_Event)+'.csv')

In [42]:
# SELECT * WHERE {
#   ?wspd rdf:type brick:Weather_Wind_Speed_Sensor .
#   ?wspd bf:hasSite ?site .
# };